In [1]:
import xaibenchmark as xb
from xaibenchmark import explainers

from xaibenchmark import load_adult as la
from xaibenchmark.comparator import ExplainerComparator
from anchor import utils

import sklearn
import sklearn.ensemble

import math
import numpy as np
import pandas as pd

from lime import lime_tabular
from anchor import anchor_tabular

In [2]:
data = la.load_csv_data('adult', root_path='../data')

In [3]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
rf.fit(xb.utils.onehot_encode(data.data, data), data.target.to_numpy().reshape(-1))
print('Train', sklearn.metrics.accuracy_score(data.target, rf.predict(xb.utils.onehot_encode(data.data, data))))
print('Dev', sklearn.metrics.accuracy_score(data.target_dev, rf.predict(xb.utils.onehot_encode(data.data_dev, data))))
print('Test', sklearn.metrics.accuracy_score(data.target_test, rf.predict(xb.utils.onehot_encode(data.data_test, data))))

Train 0.9996928881674199
Dev 0.8526822276822277
Test 0.8518518518518519


In [4]:
pred_fn = lambda x: rf.predict_proba(xb.utils.onehot_encode(x, data))
ex_lime = explainers.LimeExplainer(data, pred_fn, discretize_continuous=False)

In [5]:
pred_fn = lambda x: rf.predict(xb.utils.onehot_encode(x, data))
ex_anchors = explainers.AnchorsExplainer(data, pred_fn)

In [6]:
ex_lime.explain_instance(data.data.iloc[[0]])
ex_anchors.explain_instance(data.data.iloc[[0]])

In [7]:
comp = ExplainerComparator()
comp.add_explainer(ex_anchors, 'ANCHORS')
comp.add_explainer(ex_lime, 'LIME')

In [8]:
comp.explain_instances(data.data.iloc[[1, 2, 3000]])

In [9]:
comp.print_metrics()

Average metric values for explainer ANCHORS :
	 precision : 0.8226937661941259
	 coverage : 0.11746666666666666
	 balance_explanation : 0.6666666666666666
	 relative_area : 0.04761904799906667
	 inverse_coverage : 68.97526194513283
	 accuracy : 0.9847462203023758
	 balance_model : 0.6819204463642908
Average metric values for explainer LIME :
	 area_norm : 19.537233873073223
	 furthest_distance : 2.812814099636249
	 coverage : 6.141227152623689e-05
	 balance : 0.0925722809634537
	 accuracy : 0.1891077670044897
	 area : 2.589477453233665e+139
	 inverse_coverage : 17669.352561741227
